# Chat History with Langchain

This example will demonstrate how to create a basic chat bot using Langchain with a Writer LLM backend.

### Dependencies

Make sure you have a virtual environment selected if you don't want to install these globally.

In [159]:
%pip install -q --disable-pip-version-check\
    langchain python-dotenv

Note: you may need to restart the kernel to use updated packages.


The Writer Langchain integration looks for two environment variables: `WRITER_ORG_ID` and `WRITER_API_KEY`. If you're running this notebook, make sure you have a `.env` file in this directory that looks like this:
```
WRITER_ORG_ID=<your org ID>
WRITER_API_KEY=<your API key>
```
and run the following cell. You can alternatively use whatever method you want to set environment variables as long as they get set. 

Yet another alternative if you don't want environment variables is passing these parameters to the Langchain `Writer` object on construction.

In [160]:
from dotenv import load_dotenv
load_dotenv()

True

In [161]:
from langchain import ConversationChain
from langchain.schema import BaseOutputParser
from langchain.llms import Writer
import json
import copy

### Conversation chain

First we need to create our chain. For this example we'll have a pretty standard `ConversationChain` with a Writer llm, and set verbose mode just so we can see what's going on.

In [162]:
conversation = ConversationChain(
    llm = Writer(max_tokens=500, temperature=0),
    verbose=True
)

And now we can try it out:

In [163]:
conversation.memory.clear()
conversation.predict(input="Hello!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


'{"choices":[{"text":" Hi there! It\'s nice to meet you. How can I help you today?","logprobs":null}]}'

And it works! Kind of. But it also includes all the extra information that came with the API response. We don't want any of that for a chat bot, we just want the text. So we can create a simple parser:

In [164]:

class WriterParser(BaseOutputParser[str]):
    ignore_chain = True
    ignore_llm = True

    def parse(self, text: str) -> str:
        return json.loads(text)['choices'][0]['text']

conversation.prompt.output_parser = WriterParser()

And test again with the `predict_and_parse` method:

In [165]:
conversation.memory.clear()
conversation.predict_and_parse(input="Hello!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

Looks good! But there's still a slight problem. What if we actually try to take advantage of the history now and ask multiple questions?

In [166]:
conversation.memory.clear()
conversation.predict_and_parse(input="Hello!")
conversation.predict_and_parse(input="Tell me about yourself")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI: {"choices":[{"text":" Hi there! It's nice to meet you. How can I help you today?","logprobs":null}]}
Human: Tell me about yourself
AI:

> Finished chain.


' {"choices":[{"text":"Sure! I\'m an AI created to help people with their everyday tasks. I\'m programmed to understand natural language and provide helpful information. I\'m also constantly learning and growing, so I\'m always up for a challenge! What else would you like to know?","logprobs":null}]}'

The extra data still gets put back into the message history. The AI sees this, and thinks it should add the extra characters to the beginning and end of its response. Not only does it make the responses less readable, but it will continue to add more and more copies making each back-and-forth add extra garbage. This is obviously no good, so how do we fix it? One way is to manually keep track of the message history:

In [167]:
def chat_message(message):
    manual_memory = copy.deepcopy(conversation.memory.chat_memory)
    response = conversation.predict_and_parse(input=message)
    manual_memory.add_user_message(message)
    manual_memory.add_ai_message(response)
    conversation.memory.chat_memory = manual_memory
    return response

I know this is pretty hacky, but it's the best I could do with the limitation that `ConversationChain` appends messages to history before they are run through the parser. I believe this is because parsers often return types that are not strings. I couldn't find a way to change this, or even disable automatic message appending, but perhaps that will change in the future. For now, at least overwriting the message history with a manual copy works:

In [168]:
conversation.memory.clear()

chat_message("Hello!")
chat_message("How do I bake an apple pie?")
chat_message("thanks!")
chat_message("what about a cherry one instead?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: How do I bake an apple pie?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of sp

" Sure! To make a cherry pie, you'll need a 9-inch pie dish, a pastry blender, and the following ingredients: 4 cups of pitted and halved cherries, 1/2 cup of sugar, 1/4 teaspoon of salt, 1 teaspoon of ground cinnamon, 2 tablespoons of all-purpose flour, 2 tablespoons of butter, and a 9-inch unbaked pie crust. Preheat your oven to 425 degrees Fahrenheit. In a large bowl, combine the cherries, sugar, salt, cinnamon, and flour. Place the mixture in the pie dish and dot with butter. Place the pie crust over the top of the dish and crimp the edges. Bake for 40-45 minutes, or until the crust is golden brown. Enjoy!"

That's all for this example. To see more detail about managing memory with LLMs, take a look at the [Langchain memory documentation](https://python.langchain.com/en/latest/modules/memory.html).